<a href="https://colab.research.google.com/github/Xenosians/tubes_pemop/blob/main/Tubes_Pemop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import mean_squared_error

In [ ]:
file_path = '/content/DataClassification.txt'
df = pd.read_csv(file_path, sep='\t')

# df['x'] = np.where(df['x'] < 0, 0, df['x'])
# df['Y'] = np.where(df['Y'] < 0, 0, df['Y'])

X = df[['x', 'Y']]
y = df['LABEL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
svm_model = SVC()

param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.1, 1, 10],
}

grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

best_model = SVC(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

In [ ]:
plt.figure(figsize=(12, 8))

plt.scatter(X['x'][y == 1], X['Y'][y == 1], c='red', marker='^', label='Tipe A')
plt.scatter(X['x'][y == 2], X['Y'][y == 2], c='blue', marker='o', label='Tipe B')

ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()
xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 50), np.linspace(ylim[0], ylim[1], 50))
Z = best_model.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
contour = ax.contour(xx, yy, Z, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])

plt.xlabel('x')
plt.ylabel('Y')
plt.title('Support Vector Machine (SVM) - Decision Boundary')

plt.legend(['Tipe A', 'Tipe B', 'Decision Boundary'])

plt.subplots_adjust(bottom=0.2)

table_data = [['Metric', 'Value'],
              ['Accuracy', f'{accuracy_score(y_test, y_pred):.2f}'],
              ['F1 Score', f'{f1_score(y_test, y_pred, average="weighted"):.2f}']]
table = plt.table(cellText=table_data, loc='right', cellLoc='center', colWidths=[0.1, 0.1])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

plt.show()

In [ ]:
logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)

y_pred = logreg_model.predict(X_test)

In [ ]:
plt.figure(figsize=(12, 8))

plt.scatter(X['x'][y == 1], X['Y'][y == 1], c='red', marker='^', label='Tipe A')
plt.scatter(X['x'][y == 2], X['Y'][y == 2], c='blue', marker='o', label='Tipe B')

ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()
xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 50), np.linspace(ylim[0], ylim[1], 50))
Z = logreg_model.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
contour = ax.contour(xx, yy, Z, colors='k', levels=[0], alpha=0.5, linestyles=['-'])

plt.xlabel('x')
plt.ylabel('Y')
plt.title('Logistic Regression - Decision Boundary')

plt.legend(['Tipe A', 'Tipe B', 'Decision Boundary'])

plt.subplots_adjust(bottom=0.2)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

table_data = [['Metric', 'Value'],
              ['Accuracy', f'{accuracy:.2f}'],
              ['F1 Score', f'{f1:.2f}']]
table = plt.table(cellText=table_data, loc='right', cellLoc='center', colWidths=[0.1, 0.1])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

plt.show()

In [ ]:
param_grid_elastic_net = {
    'alpha': [1.0, 0.5, 2],
    'l1_ratio': [0.001, 0.01, 0.1],
}

grid_search_elastic_net = GridSearchCV(ElasticNet(), param_grid_elastic_net, cv=5, scoring='neg_mean_squared_error')
grid_search_elastic_net.fit(X_train, y_train)

best_params_elastic_net = grid_search_elastic_net.best_params_

best_model_elastic_net = ElasticNet(**best_params_elastic_net)
best_model_elastic_net.fit(X_train, y_train)

y_pred_elastic_net = best_model_elastic_net.predict(X_test)

mse_elastic_net = mean_squared_error(y_test, y_pred_elastic_net)

In [ ]:
plt.figure(figsize=(12, 8))

plt.scatter(X['x'][y == 1], X['Y'][y == 1], c='red', marker='^', label='Tipe A')
plt.scatter(X['x'][y == 2], X['Y'][y == 2], c='blue', marker='o', label='Tipe B')

plt.xlabel('x')
plt.ylabel('Y')
plt.title('Elastic Net Regression')

plt.legend(['Tipe A', 'Tipe B'])

plt.axhline(y=mse_elastic_net, color='black', linestyle='--', label=f'MSE: {mse_elastic_net:.2f}')

plt.subplots_adjust(bottom=0.2)

table_data = [['Metric', 'Value'],
              ['MSE', f'{mse_elastic_net:.2f}']]
table = plt.table(cellText=table_data, loc='right', cellLoc='center', colWidths=[0.1, 0.1])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

plt.show()